In [ ]:
from fiat_toolbox.well_being import Household
import matplotlib.pyplot as plt

In [ ]:
v = 0.7 # fraction of damage to the building
k_str = 30000 # building value
c0 = 48000 # annual income
c_avg = 48000 # average annual income

In [ ]:
# Create a WellBeing object and optimize the lambda value
household = Household(v=v, k_str=k_str, c0=c0, c_avg=c_avg, t_max=10, dt=1/54)
household.opt_lambda(method="discrete")
household.plot_opt_lambda(x_type="time");

In [ ]:
# Create a WellBeing object and optimize the lambda value
household = Household(v=v, k_str=k_str, c0=c0, c_avg=c_avg, t_max=10, dt=1)
household.opt_lambda(method="scipy")
household.plot_opt_lambda(x_type="time");

In [ ]:
# Create a WellBeing object and optimize the lambda value
household = Household(v=v, k_str=k_str, c0=c0, c_avg=c_avg, t_max=10, dt=1/54)
household.opt_lambda(method="discrete")
l = household.l_opt["lambda"]
z1 = household.l_opt["Utility Loss"]

household.opt_lambda(method="scipy")
z2 = household.l_opt["Utility Loss"]

# Plot the results
plt.plot(l, z1, label='Discrete Method')
plt.plot(l, z2, label='Scipy Method')
plt.xlabel('Lambda')
plt.ylabel('Utility Loss')
plt.legend()
plt.show()

In [ ]:
household = Household(v=v, k_str=k_str, c0=c0, c_avg=c_avg, t_max=10, dt=1/54)
household.opt_lambda(method="discrete")
household.get_losses("discrete")

In [ ]:
household.plot_consumption()